In [2]:
!pip install tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 916.7 kB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import os
from google.colab import drive

In [4]:
drive.mount('/content/drive')

# List contents of Google Drive root directory
folder_path = '/content/drive/My Drive/Recommendation System/Data/Data2/'
contents = os.listdir(folder_path)
print(contents)

Mounted at /content/drive
['movies_metadata.csv', 'links.csv', 'keywords.csv', 'links_small.csv', 'ratings.csv', 'ratings_small.csv', 'credits.csv']


In [5]:
movies = pd.read_csv(folder_path + 'movies_metadata.csv')
ratings = pd.read_csv(folder_path + 'ratings.csv')
# links = pd.read_csv(folder_path + 'links.csv')
# keywords = pd.read_csv(folder_path + 'keywords.csv')
# credits = pd.read_csv(folder_path + 'credits.csv')

<ipython-input-5-f5cb0a87ab94>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(folder_path + 'movies_metadata.csv')


In [6]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
movie = movies[movies['id'] == 10194]
movie

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [8]:
unique_ids_count = movies['id'].nunique()
print(unique_ids_count)


45436


In [9]:
movies.shape

(45466, 24)

In [10]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [11]:
# Check for non-numeric values in 'id' column
non_numeric_ids = pd.to_numeric(movies['id'], errors='coerce').isna()

# Drop rows with non-numeric 'id' values
movies = movies[~non_numeric_ids]

In [12]:
movies['id'] = movies['id'].astype('int64')

In [13]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [14]:
movie = ratings[ratings['movieId'] == 8844]
movie

,userId,movieId,rating,timestamp
689294,7016,8844,4.0,1257352229
701064,7144,8844,4.5,1194102660
847707,8659,8844,4.0,1153855942
939377,9547,8844,4.0,1225732406
1102121,11220,8844,0.5,1320469651
...,...,...,...,...
24758533,257389,8844,4.5,1317630819
24953358,259476,8844,4.0,1446588535
25325647,263655,8844,4.0,1500914862
25574156,266163,8844,4.5,1158843928


In [15]:
ratings.shape

(26024289, 4)

In [16]:
# credits.head()

In [17]:
# credits.shape

In [18]:
# prompt: merge these 2 dataframes
ratings['id'] = ratings['movieId']
merged_df = movies.merge(ratings, on='id', how='left')
merged_df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,userId,movieId,rating,timestamp
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1923.0,862.0,3.0,858335006.0
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,2103.0,862.0,5.0,946044912.0
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,5380.0,862.0,1.0,878941641.0
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,6177.0,862.0,4.0,859415226.0
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,6525.0,862.0,4.0,857388995.0


In [19]:
merged_df.shape

(11475531, 28)

In [20]:
unique_ids_count =merged_df['id'].nunique()
print(unique_ids_count)


45433


In [21]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11475531 entries, 0 to 11475530
Data columns (total 28 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   adult                  object 
 1   belongs_to_collection  object 
 2   budget                 object 
 3   genres                 object 
 4   homepage               object 
 5   id                     int64  
 6   imdb_id                object 
 7   original_language      object 
 8   original_title         object 
 9   overview               object 
 10  popularity             object 
 11  poster_path            object 
 12  production_companies   object 
 13  production_countries   object 
 14  release_date           object 
 15  revenue                float64
 16  runtime                float64
 17  spoken_languages       object 
 18  status                 object 
 19  tagline                object 
 20  title                  object 
 21  video                  object 
 22  vote_average    

In [22]:
merged_df = merged_df[['id', 'userId', 'genres', 'title', 'rating', 'overview']]

In [23]:
merged_df.shape

(11475531, 6)

In [24]:
merged_df.isnull().sum()

id              0
userId      37894
genres          0
title           3
rating      37894
overview    41483
dtype: int64

In [25]:
merged_df1 = merged_df.dropna()
merged_df1.shape

(11396980, 6)

In [26]:
merged_df1.isnull().sum()

id          0
userId      0
genres      0
title       0
rating      0
overview    0
dtype: int64

In [27]:
unique_ids_count =merged_df1['id'].nunique()
print(unique_ids_count)

7437


In [28]:
merged_df1.head()

,id,userId,genres,title,rating,overview
0,862,1923.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,3.0,"Led by Woody, Andy's toys live happily in his ..."
1,862,2103.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,5.0,"Led by Woody, Andy's toys live happily in his ..."
2,862,5380.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,1.0,"Led by Woody, Andy's toys live happily in his ..."
3,862,6177.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."
4,862,6525.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."


In [29]:
# prompt: check for duplicates using id, userId, genres, title, rating columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
duplicate_rows = merged_df1[merged_df1.duplicated(subset=['id', 'userId', 'genres', 'title', 'rating', 'overview'])]
duplicate_rows.head()


,id,userId,genres,title,rating,overview
7287508,5511,32711.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Le Samouraï,2.0,Hitman Jef Costello is a perfectionist who alw...
7287509,5511,37732.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Le Samouraï,0.5,Hitman Jef Costello is a perfectionist who alw...
7287510,5511,40397.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Le Samouraï,4.0,Hitman Jef Costello is a perfectionist who alw...
7287511,5511,41192.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Le Samouraï,2.0,Hitman Jef Costello is a perfectionist who alw...
7287512,5511,45018.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Le Samouraï,2.0,Hitman Jef Costello is a perfectionist who alw...


In [30]:
merged_df1.drop_duplicates(inplace=True)

<ipython-input-30-5c2d7f5941b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df1.drop_duplicates(inplace=True)


In [31]:
merged_df1.shape

(11395911, 6)

In [32]:
unique_ids_count =merged_df1['id'].nunique()
print(unique_ids_count)

7437


In [33]:
merged_df1 = merged_df1.head(500000)

In [34]:
unique_ids_count =merged_df1['id'].nunique()
print(unique_ids_count)

133


In [35]:
merged_df1['userId'] = merged_df1['userId'].astype(int)

In [36]:
merged_df1.head()

,id,userId,genres,title,rating,overview
0,862,1923,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,3.0,"Led by Woody, Andy's toys live happily in his ..."
1,862,2103,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,5.0,"Led by Woody, Andy's toys live happily in his ..."
2,862,5380,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,1.0,"Led by Woody, Andy's toys live happily in his ..."
3,862,6177,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."
4,862,6525,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."


In [37]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
items = merged_df1[['id', 'overview']].drop_duplicates().reset_index(drop=True)
inputs = tokenizer(items['overview'].tolist(), return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state.mean(dim=1)

item_embeddings_df = pd.DataFrame(embeddings.numpy(), index=items['id'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [38]:
item_embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
862,-0.201149,0.095459,0.270211,-0.042554,0.215916,0.016117,0.153031,0.276025,0.214446,-0.097193,-0.144763,-0.254026,-0.062525,0.325143,-0.120101,0.123275,0.295998,0.137442,0.031656,0.322005,0.117975,-0.001547,-0.224415,0.217604,0.485744,0.231055,-0.135742,-0.011129

In [39]:
item_embeddings_df.shape

(133, 768)

In [40]:
merged_df1.head()

,id,userId,genres,title,rating,overview
0,862,1923,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,3.0,"Led by Woody, Andy's toys live happily in his ..."
1,862,2103,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,5.0,"Led by Woody, Andy's toys live happily in his ..."
2,862,5380,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,1.0,"Led by Woody, Andy's toys live happily in his ..."
3,862,6177,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."
4,862,6525,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,4.0,"Led by Woody, Andy's toys live happily in his ..."


In [46]:
# merged_df1.set_index('userId', inplace=True)
# merged_df1.head()

KeyError: "None of ['userId'] are in the columns"

In [42]:
user_item_matrix = merged_df1.pivot(index='userId', columns='id', values='rating').fillna(0)

pd.set_option('display.float_format', '{:.2f}'.format)
user_item_df = pd.DataFrame(user_item_matrix, index=user_item_matrix.index, columns=user_item_matrix.columns)
user_item_df.head()

id,5,6,11,13,63,76,101,103,108,109,110,178,193,197,236,241,278,281,306,315,329,397,400,406,414,451,507,522,524,527,568,577,628,629,649,680,687,688,710,712,755,807,854,862,880,888,902,949,1024,1408,1413,1572,1587,1642,1689,1710,1873,1909,1945,2019,2045,2054,2064,2074,2086,2087,2124,2164,2246,2259,2292,2293,2307,2320,2636,2654,2758,2759,2788,3036,3049,3512,3586,4476,4482,4584,4722,4916,4954,5503,5757,5894,5967,6071,6075,6280,6520,6715,6950,7984,8011,8012,8068,8391,8447,8512,8587,8831,8839,8844,8963,8973,8984,8986,8987,26141,26203,26258,26271,26391,26564,27526,27793,32325,32646,49299,65796,95963,117553,124639,139405,159185,171857
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [43]:
user_item_df.shape

(159574, 133)

In [44]:
from scipy.sparse.linalg import svds

matrix = user_item_df.to_numpy()

# Normalize by subtracting the mean
user_ratings_mean = np.mean(matrix, axis=1)
matrix_demeaned = matrix - user_ratings_mean.reshape(-1, 1)

# Perform SVD
U, sigma, Vt = svds(matrix_demeaned, k=50)

# Convert sigma to diagonal matrix form
sigma = np.diag(sigma)
sigma

array([[  94.62819448,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   97.37561533,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,   99.9899731 , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,  625.48550788,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         788.06707799,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1426.01823296]])

In [45]:
reconstructed_matrix = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
reconstructed_df = pd.DataFrame(reconstructed_matrix, columns=user_item_matrix.columns, index=user_item_matrix.index)
reconstructed_df.head()


id,5,6,11,13,63,76,101,103,108,109,110,178,193,197,236,241,278,281,306,315,329,397,400,406,414,451,507,522,524,527,568,577,628,629,649,680,687,688,710,712,755,807,854,862,880,888,902,949,1024,1408,1413,1572,1587,1642,1689,1710,1873,1909,1945,2019,2045,2054,2064,2074,2086,2087,2124,2164,2246,2259,2292,2293,2307,2320,2636,2654,2758,2759,2788,3036,3049,3512,3586,4476,4482,4584,4722,4916,4954,5503,5757,5894,5967,6071,6075,6280,6520,6715,6950,7984,8011,8012,8068,8391,8447,8512,8587,8831,8839,8844,8963,8973,8984,8986,8987,26141,26203,26258,26271,26391,26564,27526,27793,32325,32646,49299,65796,95963,117553,124639,139405,159185,171857
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,1.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,3.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.01,-0.00,-0.00,-0.00,0.01,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.01,0.01,-0.00,-0.00,-0.00,0.00,-0.01,0.02,4.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.01,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.01,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.02,0.01,-0.00,0.00,-0.01,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.03,-0.00,0.01,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.01,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,4.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.02,-0.00,-0.00,-0.00,0.00,-0.01,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.01,-0.00,-0.00,-0.00,4.00,0.00,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.02,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.02,-0.00,-0.00,0.01,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
5,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.01,-0.01,0.00,-0.02,-0.00,-0.00,0.00,-0.01,0.01,-0.00,0.00,0.00,0.00,-0.01,-0.01,0.00,0.00,0.01,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,-0.01,-0.00,-0.00,0.00,-0.0

In [46]:
reconstructed_df.head().index

Index([1, 2, 3, 4, 5], dtype='int64', name='userId')

In [47]:
reconstructed_df.shape

(159574, 133)

In [48]:
# Ensure the indices and columns align
print("Original matrix indices:", user_item_matrix.index)
print("Reconstructed matrix indices:", reconstructed_df.index)
print("Original matrix columns:", user_item_matrix.columns)
print("Reconstructed matrix columns:", reconstructed_df.columns)

Original matrix indices: Index([     1,      2,      3,      4,      5,      6,      8,     11,     12,
           15,
       ...
       270883, 270884, 270885, 270887, 270891, 270892, 270893, 270894, 270895,
       270896],
      dtype='int64', name='userId', length=159574)
Reconstructed matrix indices: Index([     1,      2,      3,      4,      5,      6,      8,     11,     12,
           15,
       ...
       270883, 270884, 270885, 270887, 270891, 270892, 270893, 270894, 270895,
       270896],
      dtype='int64', name='userId', length=159574)
Original matrix columns: Index([     5,      6,     11,     13,     63,     76,    101,    103,    108,
          109,
       ...
        32325,  32646,  49299,  65796,  95963, 117553, 124639, 139405, 159185,
       171857],
      dtype='int64', name='id', length=133)
Reconstructed matrix columns: Index([     5,      6,     11,     13,     63,     76,    101,    103,    108,
          109,
       ...
        32325,  32646,  49299,  65796, 

In [49]:
def recommend_items(user_id, original_matrix, reconstructed_matrix, movies_df, num_recommendations):
    # Ensure user_id is correctly indexed (assuming user IDs are 1-based)
    user_index = user_id - 1

    # Sort the original ratings and predictions for the user
    sorted_user_ratings = original_matrix.loc[user_id].sort_values(ascending=False)
    sorted_user_predictions = reconstructed_df.loc[user_id].sort_values(ascending=False)

    # Filter out items already rated by the user
    rated_items = sorted_user_ratings[sorted_user_ratings != 0].index
    recommendations = sorted_user_predictions[~sorted_user_predictions.index.isin(rated_items)]

    # Get unique recommendations
    unique_recommendations = recommendations.drop_duplicates().head(num_recommendations)

    return unique_recommendations

# Example usage
user_id = 37
num_recommendations = 10

recommendations = recommend_items(user_id, user_item_matrix, reconstructed_df, movies, num_recommendations)
recommended_titles = merged_df1[merged_df1['id'].isin(recommendations.index)]
recommended_titles = recommended_titles[['title']]
recommended_titles = recommended_titles.drop_duplicates()
recommended_titles.reset_index(drop=True, inplace=True)

recommended_titles

,title
0,Sense and Sensibility
1,To Die For
2,Nick of Time
3,Unforgettable
4,Devil in a Blue Dress
5,The Net
6,Legends of the Fall
7,Outbreak
8,The Flintstones
9,Guilty as Sin


In [50]:
user = merged_df1[merged_df1['userId'] == 37]
user

,id,userId,genres,title,rating,overview
182387,527,37,"[{'id': 18, 'name': 'Drama'}]",Once Were Warriors,4.00,A drama about a Maori family lving in Auckland...
413033,2087,37,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",The Getaway,3.00,Doc McCoy is put in prison because his partner...


In [55]:
user = merged_df1[merged_df1['title'] == 'Outbreak']
pd.set_option('display.max_colwidth', None)
user.head(1)

,id,userId,genres,title,rating,overview
250040,6950,46,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'name': 'Drama'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 53, 'name': 'Thriller'}]",Outbreak,2.50,"A deadly airborne virus finds its way into the USA and starts killing off people at an epidemic rate. Col Sam Daniels' job is to stop the virus spreading from a small town, which must be quarantined, and to prevent an over reaction by the White House."


In [56]:
import numpy as np
from sklearn.model_selection import train_test_split

# Create a train-test split
train_data, test_data = train_test_split(merged_df1, test_size=0.2, random_state=42)

# Create user-item matrices for training and testing
train_matrix = train_data.pivot(index='userId', columns='id', values='rating').fillna(0)
test_matrix = test_data.pivot(index='userId', columns='id', values='rating').fillna(0)


In [57]:
from sklearn.metrics import mean_squared_error

def calculate_rmse(original_matrix, reconstructed_matrix, test_data):
    # Flatten the matrices and filter out the zero values (unrated items)
    actual = []
    predicted = []

    for index, row in test_data.iterrows():
        user_id = row['userId']
        item_id = row['id']
        actual_rating = row['rating']

        # Get the predicted rating from the reconstructed matrix
        predicted_rating = reconstructed_matrix.loc[user_id, item_id]

        actual.append(actual_rating)
        predicted.append(predicted_rating)

    # Calculate RMSE
    actual = np.array(actual)
    predicted = np.array(predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))

    return rmse

# Calculate RMSE
rmse = calculate_rmse(user_item_matrix, reconstructed_df, test_data)
print(f"RMSE: {rmse}")


RMSE: 0.5876299066452203
